In [1]:
import pandas as pd
import numpy as np
import datetime as dt

## EDA Problem 1
Meteorologia

Generacion_fotovoltaica

Consumo_fotovoltaica

In [2]:
genfot = pd.read_csv("datasets/Generacion_fotovoltaica.csv", sep=",")

In [3]:
(genfot["TOTAL_KWH_ENERGIA"].isna().sum())/len(genfot["TOTAL_KWH_ENERGIA"])

np.float64(0.11990111248454882)

In [4]:
genfot["y"] = genfot["TOTAL_KWH_ENERGIA"]
genfot.drop(columns=["TOTAL_KWH_ENERGIA"], inplace=True)

In [5]:
genfot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9708 entries, 0 to 9707
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   FECHA   9708 non-null   object 
 1   y       8544 non-null   float64
dtypes: float64(1), object(1)
memory usage: 151.8+ KB


In [6]:
genfot['FECHA'] = pd.to_datetime(genfot['FECHA'])

See how many negative values there are

In [7]:
counter = 0
sum = 0
indexes = []
for i, v in enumerate(genfot["y"]):
    if v < 0:
        sum += v
        counter += 1
        indexes.append(i)
aux = genfot.loc[indexes].copy()
print(counter)

212


See which hours tend to have the most negative values

In [ ]:
genfot.loc[genfot['FECHA'].dt.hour.isin([22, 23, 0, 1, 2, 3, 4, 5, 6, 7]) & genfot['y'].isnull(), 'y'] = 0

print(genfot['y'].isnull().sum())

699


Mean imputation

We need to see how we treat and interpret negative values. The base code for imputing mean energy grouped by hour is here!

In [10]:
aux = genfot.dropna()
hourly_mean = aux.groupby(aux['FECHA'].dt.hour)['y'].mean()
aux = pd.DataFrame(hourly_mean, columns=["y"])
aux

,y
FECHA,
0,0.002772
1,-0.003119
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.020693
8,1.183267


In [11]:
for i, v in enumerate(genfot["y"]):
    if pd.isna(v):
        aux = genfot["FECHA"][i].hour
        genfot["y"][i] = hourly_mean[aux]

C:\Users\dario\AppData\Local\Temp\ipykernel_17924\3636442234.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  genfot["y"][i] = hourly_mean[aux]
C:\Users\dario\AppData\Local\Temp\ipykernel_17924\3636442234.py:4: SettingWithCopyWarning: 
A 

In [12]:
genfot.head()

,FECHA,y
0,2023-07-24 11:00:00,50.621836
1,2023-07-24 12:00:00,115.660000
2,2023-07-24 13:00:00,129.980000
3,2023-07-24 14:00:00,134.350000
4,2023-07-24 15:00:00,133.900000


As you can see, there are quite a lot of changes in between the mean and some values, as seen in the previous cell. What are the next steps?

In [13]:
genfot.isna().sum()

genfot.to_csv("datasets/Generacion_fotovoltaica_imputed.csv", index=False)